<a href="https://colab.research.google.com/github/swha03/G_project/blob/master/3.%20Data%20modeling/cnnmodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **main**

## initial settings

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

In [0]:
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D
import keras
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow
import os
import pickle
import cv2
import numpy as np
import pandas as pd
import glob

## global var

In [0]:
filepath = '/content/drive/My Drive/Colab Notebooks/sample.dat'
modelpath = '/content/drive/My Drive/Colab Notebooks/mymodel.h5'

## methods

### collect_data

In [0]:
def collect_data(dirname, csvname):
    with open(filepath, 'ab') as f:
        data = []
        csv = pd.read_csv(csvname, sep=',')
        joy_values = csv['wheel'].values.tolist()

        images = glob.glob(dirname)

        count = 0

        for img in images:

            screenshot = cv2.imread(img)

            if count < len(joy_values):
                screenshot = np.array(screenshot)

                data.append([screenshot, joy_values[count]])

            if count == len(images) - 1:
                count += 1
                print('Collected data count - {0}.'.format(count))
                pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
                data = [] 

            count += 1

### read_data

In [0]:
def read_data(split=False):
    with open(filepath, 'rb') as f:
        data = []
        while True:
            try:
                temp = pickle.load(f)

                if type(temp) is not list:
                    temp = np.ndarray.tolist(temp)

                data = data + temp
            except EOFError:
                break
        if split:
            x_train = []
            y_train = []

            for i in range(0, len(data)):
                x_train.append(data[i][0])
                y_train.append(data[i][1])

            return np.array(x_train), np.array(y_train)
        else:
            return np.array(data)

### create_model

In [0]:
def create_model():
    nrows = 66
    ncols = 200
    img_channels = 3
    output_size = 1

    model = Sequential()
    model.add(Dropout(0.35, input_shape=(nrows, ncols, img_channels)))
    model.add(Conv2D(filters=24, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='elu'))
    model.add(Conv2D(filters=36, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='elu'))
    model.add(Conv2D(filters=48, kernel_size=(5, 5), strides=(2, 2), padding='valid', activation='elu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='elu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=(1, 1), padding='valid', activation='elu'))
    model.add(Dropout(0.35))

    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(output_size))
    model.summary()

    model.compile(loss=keras.losses.mean_squared_error,
                  optimizer=keras.optimizers.Adam(lr=0.001),
                  metrics=['accuracy'])

    return model

### get_model

In [0]:
def get_model():
    if os.path.isfile(modelpath):
        model = keras.models.load_model(modelpath)
    else:
        model = create_model()

    return model

### train_model

In [0]:
def train_model(model):
    x, y = read_data(True)

    # test data set 0.15 : training data set 85%
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15, random_state=1)

    model = get_model()

    checkpoint = ModelCheckpoint('model-{epoch:03d}-{val_loss:4f}.h5', monitor='val_loss', verbose=1, save_best_only=True)
    earlystop = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)

    model.fit(x_train, y_train, epochs=100, batch_size=100, verbose=1,
              validation_data=(x_valid, y_valid), callbacks=[checkpoint,earlystop])

    model.save(modelpath)

## training section

In [0]:
model = get_model()
train_model(model)

## testing section

In [0]:
model = get_model()

images = glob.glob('/content/drive/My Drive/Colab Notebooks/CFile1/*.jpg')
